In [4]:
import keras
from keras.models import load_model
from keras.models import Model, Sequential
print("keras version:", keras.__version__)
import tensorflow as tf
print("tensorflow version:", tf.__version__)
import wget
import os
import sys
from keras.datasets import cifar10
import numpy as np
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
x_valid = x_train[:10000]
y_valid = y_train[:10000]

keras version: 2.3.1
tensorflow version: 1.15.0


In [27]:
model_files = [
    "model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-20_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-30_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-40_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-50_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-60_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-70_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-80_bestbefore-100_currentepoch-100_valacc-90_vgg.h5",
    "model_cifar10_balanced_seed-90_bestbefore-100_currentepoch-100_valacc-91_vgg.h5"
]
model_file = model_files[0] # test with 1 model

print("On model", model_file)
#   if (os.path.isfile(model_file)==False):
#     print("Downloading", model_file)
#     wget.download("https://zenodo.org/record/2648107/files/"
#                   +model_file+"?download=1", out=model_file)
model = load_model("models/" + model_file)

model without dropout layers
trimmed_model = Sequential()

for layer in model.layers[:-1]:
    if 'dropout' not in layer.name:
        trimmed_model.add(layer)

print(trimmed_model.layers)

# print(model.input)
# print(len(model.layers))
# print(model.layers[-2])
# print(model.layers[-2].output)
pre_softmax_model = Model(input=model.input, output=model.layers[-2].output)

print("Making predictions on validation set")
valid_preacts = pre_softmax_model.predict(x_valid)
print("Making predictions on test set")
test_preacts = pre_softmax_model.predict(x_test)

print("Test accuracy",np.mean(np.argmax(test_preacts,axis=1)
                            == np.squeeze(y_test)))
print("Valid accuracy",np.mean(np.argmax(valid_preacts,axis=1)
                             == np.squeeze(y_valid)))


On model model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5


/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on validation set
Making predictions on test set
Test accuracy 0.8967
Valid accuracy 0.9054


In [14]:
print(trimmed_model.weights)

[<tf.Variable 'conv2d_1_16/kernel:0' shape=(3, 3, 3, 64) dtype=float32>, <tf.Variable 'conv2d_1_16/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'batch_normalization_1_16/gamma:0' shape=(64,) dtype=float32>, <tf.Variable 'batch_normalization_1_16/beta:0' shape=(64,) dtype=float32>, <tf.Variable 'conv2d_2_16/kernel:0' shape=(3, 3, 64, 64) dtype=float32>, <tf.Variable 'conv2d_2_16/bias:0' shape=(64,) dtype=float32>, <tf.Variable 'batch_normalization_2_16/gamma:0' shape=(64,) dtype=float32>, <tf.Variable 'batch_normalization_2_16/beta:0' shape=(64,) dtype=float32>, <tf.Variable 'conv2d_3_16/kernel:0' shape=(3, 3, 64, 128) dtype=float32>, <tf.Variable 'conv2d_3_16/bias:0' shape=(128,) dtype=float32>, <tf.Variable 'batch_normalization_3_16/gamma:0' shape=(128,) dtype=float32>, <tf.Variable 'batch_normalization_3_16/beta:0' shape=(128,) dtype=float32>, <tf.Variable 'conv2d_4_16/kernel:0' shape=(3, 3, 128, 128) dtype=float32>, <tf.Variable 'conv2d_4_16/bias:0' shape=(128,) dtype=float32>, 

In [3]:
from deepexplain.tensorflow import DeepExplain
from keras import backend as K

with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    # Need to reconstruct the graph in DeepExplain context, using the same weights.
    # With Keras this is very easy:
    # 1. Get the input tensor to the original model
    input_tensor = model.input # model.layers[0].input
    print(input_tensor)
    
    # 2. We now target the output of the last dense layer (pre-softmax)
    # To do so, create a new model sharing the same layers untill the last dense (index -2)
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    print(target_tensor)
    #y_train = keras.utils.to_categorical(y_train, num_classes)
    num_classes = 10
    y_test = keras.utils.to_categorical(y_test, num_classes)
    xs = x_test[0:1]
    ys = y_test[0:1]
    
    print(np.array(xs).shape)
    print(np.array(ys).shape)
    
    #attributions_gradin = de.explain('grad*input', target_tensor, input_tensor, xs, ys=ys)
    #attributions_sal   = de.explain('saliency', target_tensor, input_tensor, xs, ys=ys)
    attributions_ig    = de.explain('intgrad', target_tensor, input_tensor, xs, ys=ys, steps=100)
    attributions_dl    = de.explain('deeplift', target_tensor, input_tensor, xs, ys=ys)
    attributions_idl   = de.explain('idl_true', target_tensor, input_tensor, xs, ys=ys, steps=100)


Tensor("conv2d_1_input:0", shape=(?, 32, 32, 3), dtype=float32)
Tensor("model_2/dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)
(1, 32, 32, 3)
(1, 10)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `tf.variables_initializer` instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


...

DeepLIFT: references ready
{'activation_1/Relu': <tf.Variable 'Variable:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_2/Relu': <tf.Variable 'Variable_1:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_3/Relu': <tf.Variable 'Variable_2:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_4/Relu': <tf.Variable 'Variable_3:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_5/Relu': <tf.Variable 'Variable_4:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_6/Relu': <tf.Variable 'Variable_5:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_7/Relu': <tf.Variable 'Variable_6:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_8/Relu': <tf.Variable 'Variable_7:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_9/Relu': <tf.Variable 'Variable_8:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_10/Relu': <tf.Variable 'Variable_9:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_11/Relu': <tf.Variable 'Variable_10:0' shape=(1, 2, 2, 

DeepLIFT: references ready
{'activation_1/Relu': <tf.Variable 'Variable:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_2/Relu': <tf.Variable 'Variable_1:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_3/Relu': <tf.Variable 'Variable_2:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_4/Relu': <tf.Variable 'Variable_3:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_5/Relu': <tf.Variable 'Variable_4:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_6/Relu': <tf.Variable 'Variable_5:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_7/Relu': <tf.Variable 'Variable_6:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_8/Relu': <tf.Variable 'Variable_7:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_9/Relu': <tf.Variable 'Variable_8:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_10/Relu': <tf.Variable 'Variable_9:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_11/Relu': <tf.Variable 'Variable_10:0' shape=(1, 2, 2, 

DeepLIFT: references ready
{'activation_1/Relu': <tf.Variable 'Variable:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_2/Relu': <tf.Variable 'Variable_1:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_3/Relu': <tf.Variable 'Variable_2:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_4/Relu': <tf.Variable 'Variable_3:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_5/Relu': <tf.Variable 'Variable_4:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_6/Relu': <tf.Variable 'Variable_5:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_7/Relu': <tf.Variable 'Variable_6:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_8/Relu': <tf.Variable 'Variable_7:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_9/Relu': <tf.Variable 'Variable_8:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_10/Relu': <tf.Variable 'Variable_9:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_11/Relu': <tf.Variable 'Variable_10:0' shape=(1, 2, 2, 

DeepLIFT: references ready
{'activation_1/Relu': <tf.Variable 'Variable:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_2/Relu': <tf.Variable 'Variable_1:0' shape=(1, 32, 32, 64) dtype=float32_ref>, 'activation_3/Relu': <tf.Variable 'Variable_2:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_4/Relu': <tf.Variable 'Variable_3:0' shape=(1, 16, 16, 128) dtype=float32_ref>, 'activation_5/Relu': <tf.Variable 'Variable_4:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_6/Relu': <tf.Variable 'Variable_5:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_7/Relu': <tf.Variable 'Variable_6:0' shape=(1, 8, 8, 256) dtype=float32_ref>, 'activation_8/Relu': <tf.Variable 'Variable_7:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_9/Relu': <tf.Variable 'Variable_8:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_10/Relu': <tf.Variable 'Variable_9:0' shape=(1, 4, 4, 512) dtype=float32_ref>, 'activation_11/Relu': <tf.Variable 'Variable_10:0' shape=(1, 2, 2, 

results
[[[[-1.62723283e-03  6.24450084e-05  1.43985680e-03]
   [-2.87584829e-06  2.00133980e-03  1.41454807e-02]
   [ 1.16321073e-02  2.29951006e-05 -1.49122895e-03]
   ...
   [ 3.42580577e-04  2.25223093e-03  1.31951201e-02]
   [-2.15648517e-04  1.18765109e-02  1.88630926e-02]
   [ 6.94397335e-04  5.36652986e-03 -7.27864050e-03]]

  [[ 1.15175441e-03 -7.65959972e-04 -2.13347888e-03]
   [-2.81201475e-03  2.86386811e-03  3.22949888e-02]
   [ 3.73559524e-03  1.32481724e-03  1.83310674e-02]
   ...
   [ 6.67243065e-03 -2.48793177e-03  1.52221229e-02]
   [ 2.00412011e-03  5.05632218e-04  6.16565070e-03]
   [ 6.23886216e-04  6.85934667e-03  7.06203945e-03]]

  [[-4.29919062e-03 -1.64839187e-03 -5.40650999e-03]
   [-2.31898729e-04 -3.56543903e-03 -3.12960086e-03]
   [ 1.01841212e-02 -1.97947509e-03 -1.59441744e-02]
   ...
   [ 4.57277389e-03 -1.27630496e-03  1.63217320e-02]
   [ 3.41052336e-03 -1.22776865e-03  3.03612519e-03]
   [-4.12947596e-05 -8.32455079e-03 -2.83235948e-02]]

  ...

  [[

In [4]:
# print(np.mean(np.abs(attributions_dl) - np.abs(attributions_idl)))

In [6]:
print(np.mean(np.abs(attributions_ig) - np.abs(attributions_idl)))
print(np.max(attributions_ig))
print(np.max(attributions_idl))

0.0004394068720059119
0.4279742148414418
0.4175996451665156


In [13]:
fModel.layers